In [6]:
# primeiro baixar libpq-dev no terminal
pip install psycopg2

     |████████████████████████████████| 380 kB 1.7 MB/s eta 0:00:01
  Created wheel for psycopg2: filename=psycopg2-2.9.3-cp39-cp39-linux_x86_64.whl size=166246 sha256=35b4f8ff0fc0d4239d293a46ea23e18cbae67a993b2cb0ff1b6e1d5eae893dc1
  Stored in directory: /home/arquinho/.cache/pip/wheels/b3/a1/6e/5a0e26314b15eb96a36263b80529ce0d64382540ac7b9544a9
Successfully built psycopg2
Note: you may need to restart the kernel to use updated packages.


In [10]:
import psycopg2 as pg
import pandas as pd

In [56]:
# Credential
host = 'comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com'
port = 5432
database = 'comunidadedsdb'
username = 'member'
pwd = 'cdspa'

In [58]:
conn = pg.connect( user=username,
                     password=pwd,
                     host=host,
                     port=port,
                     database=database)

##  Schema Query

In [39]:
cursor = conn.cursor()

query_schema = """
    SELECT nspname
    FROM pg_catalog.pg_namespace
"""
cursor.execute( query_schema )

In [40]:
record = cursor.fetchall()
cursor.close()
conn.close()

record

[('pg_toast',),
 ('pg_temp_1',),
 ('pg_toast_temp_1',),
 ('pg_catalog',),
 ('information_schema',),
 ('public',),
 ('pa004',),
 ('pg_temp_4',),
 ('pg_toast_temp_4',),
 ('pa005',),
 ('pg_temp_9',),
 ('pg_toast_temp_9',)]

## 2.0 Table Query

In [46]:
cursor = conn.cursor()

query_tables = """
    SELECT tablename
    FROM pg_tables
    WHERE schemaname='pa004'
"""

cursor.execute( query_tables )

record = cursor.fetchall()
print( record )

cursor.close()
conn.close()



[('users',), ('vehicle',), ('insurance',)]


## Collecting Data With Psycopg2

In [53]:
cursor = conn.cursor()

query_tables_users = """
    SELECT *
    FROM pa004.users u
    WHERE u.age > 44
    LIMIT 10
"""

cursor.execute( query_tables_users )

record = cursor.fetchall()
print( record )

cursor.close()
conn.close()


[(2, 'Male', 76, 3.0, 26.0), (3, 'Male', 47, 28.0, 26.0), (8, 'Female', 56, 28.0, 26.0), (11, 'Female', 47, 35.0, 124.0), (14, 'Male', 76, 28.0, 13.0), (15, 'Male', 71, 28.0, 30.0), (20, 'Female', 60, 33.0, 124.0), (21, 'Male', 65, 28.0, 124.0), (22, 'Male', 49, 28.0, 124.0), (27, 'Female', 51, 28.0, 124.0)]


In [55]:
data = pd.DataFrame( record )
data.head()

,0,1,2,3,4
0,2,Male,76,3.0,26.0
1,3,Male,47,28.0,26.0
2,8,Female,56,28.0,26.0
3,11,Female,47,35.0,124.0
4,14,Male,76,28.0,13.0


##  COLLECTING DATA USING PANDAS
**MELHOR** metodo e mais atual

In [59]:
import pandas.io.sql as psql

In [68]:
cursor = conn.cursor()

query_tables_users = """
    SELECT *
    FROM pa004.users u LEFT JOIN pa004.vehicle v ON ( u.id=v.id)
                       LEFT JOIN pa004.insurance i ON (u.id=i.id)
    WHERE u.age > 44
    LIMIT 10
"""

df = psql.read_sql( query_tables_users, conn)
df.head()

,id,gender,age,region_code,policy_sales_channel,id,driving_license,vehicle_age,vehicle_damage,id,previously_insured,annual_premium,vintage,response
0,14285,Male,71,28.0,26.0,14285,1,1-2 Year,No,14285,0,37836.0,148,0
1,39,Male,45,8.0,124.0,39,1,1-2 Year,Yes,39,0,42297.0,264,0
2,114,Male,50,30.0,26.0,114,1,1-2 Year,Yes,114,0,34271.0,229,1
3,126,Male,49,28.0,124.0,126,1,1-2 Year,Yes,126,0,2630.0,88,0
4,131,Male,50,28.0,26.0,131,1,1-2 Year,Yes,131,0,57389.0,155,1


In [70]:
df1 = pd.read_sql( query_tables_users, conn )
df1 = df1.drop(df.columns[[5, 9]], axis=1) #comeca a contar do 0
df1.head()

,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,Male,50,28.0,124.0,1,> 2 Years,Yes,0,53195.0,109,0
1,Male,45,8.0,124.0,1,1-2 Year,Yes,0,42297.0,264,0
2,Male,50,30.0,26.0,1,1-2 Year,Yes,0,34271.0,229,1
3,Male,49,28.0,124.0,1,1-2 Year,Yes,0,2630.0,88,0
4,Male,50,28.0,26.0,1,1-2 Year,Yes,0,57389.0,155,1
